In [2]:
#Importaciones
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel, delayed, effective_n_jobs
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [3]:
datosGeneracion = pd.read_excel("../Datasets/Información Planta Solar Estacionaria.xlsx")
datosGeneracion.Fecha = datosGeneracion.Fecha.values.astype(np.int64) / 10 ** 9
datosGeneracion

,Fecha,Anio,Mes,NumMes,Dia,NumDia,Hora,NumHora,Generacion,Temperatura,...,Generacion_diff,Temperatura_diff,ProbabilidadLluvia_diff,HumedadRelativa_diff,VelocidadViento_diff,DireccionViento_diff,IndiceUV_diff,VelocidadRafaga_diff,Precipitacion_diff,DPT_diff
0,1.625184e+09,2021,199,7,6027,2,144624,0,0.0000,18.0,...,0.0000,-1.0,4,1.0,-2.0,0.0,0,-2.0,0.0,0.0
1,1.625188e+09,2021,199,7,6027,2,144625,1,0.0000,18.0,...,0.0000,0.0,8,0.0,0.0,0.0,0,-2.0,0.0,0.0
2,1.625191e+09,2021,199,7,6027,2,144626,2,0.0000,18.0,...,0.0000,0.0,5,2.0,-2.0,0.0,0,-2.0,0.0,0.0
3,1.625195e+09,2021,199,7,6027,2,144627,3,0.0000,17.0,...,0.0000,-1.0,0,1.0,0.0,0.0,0,-2.0,0.0,-1.0
4,1.625198e+09,2021,199,7,6027,2,144628,4,0.0000,17.0,...,0.0000,0.0,0,7.0,0.0,0.0,0,-2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18917,1.693508e+09,2023,224,8,6817,31,163603,19,1732.7788,25.2,...,1165.2214,0.6,0,-11.0,-2.5,16.0,0,-5.4,0.0,-3.4
18918,1.693512e+09,2023,224,8,6817,31,163604,20,0.0000,21.5,...,0.0000,2.5,0,-20.0,-1.4,43.0,0,-3.6,-1.4,-2.7
18919,1.693516e+09,2023,224,8,6817,31,163605,21,0.0000,20.8,...,0.0000,2.4,0,-20.0,0.3,33.0,0,-2.8,-1.3,-2.6
18920,1.693519e+09,2023,224,8,6817,31,163606,22,0.0000,18.2,...,0.0000,0.2,0,-28.0,5.8,-9.0,0,5.0,0.0,-6.3


In [4]:
x = datosGeneracion[['Generacion_diff', 'Temperatura_diff', 'ProbabilidadLluvia_diff', 'DireccionViento_diff',
                     'IndiceUV_diff', 'CodCondCielo', 'CodDirViento', 'VelocidadRafaga_diff', 'DPT_diff']]

In [5]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(x)
x_scaled = pd.DataFrame(scaled, columns = x.columns)
x_scaled

,Generacion_diff,Temperatura_diff,ProbabilidadLluvia_diff,DireccionViento_diff,IndiceUV_diff,CodCondCielo,CodDirViento,VelocidadRafaga_diff,DPT_diff
0,0.500199,0.447368,0.527950,0.494949,0.444444,0.75,0.375,0.494208,0.507246
1,0.500199,0.473684,0.552795,0.494949,0.444444,0.75,0.375,0.494208,0.507246
2,0.500199,0.473684,0.534161,0.494949,0.444444,0.25,0.375,0.494208,0.507246
3,0.500199,0.447368,0.503106,0.494949,0.444444,0.25,0.375,0.494208,0.492754
4,0.500199,0.473684,0.503106,0.494949,0.444444,0.25,0.375,0.494208,0.521739
...,...,...,...,...,...,...,...,...,...
18917,0.519676,0.489474,0.503106,0.518038,0.444444,0.50,0.250,0.461390,0.457971
18918,0.500199,0.539474,0.503106,0.556999,0.444444,0.50,0.250,0.478764,0.468116
18919,0.500199,0.536842,0.503106,0.542569,0.444444,0.50,0.375,0.486486,0.469565
18920,0.500199,0.478947,0.503106,0.481962,0.444444,0.50,0.375,0.561776,0.415942


## Métodos para selección de cuántos clusters son más óptimos

In [6]:

# silhouette_score -> entre más alto mejor
# davies_bouldin_score -> entre más bajo mejor

#Clusters con el algoritmo K-Means (K-medias)
clusters_km = KMeans(n_clusters = 9, random_state = 1)
clusters_km.fit_predict(x_scaled)

#Cálculo de métricas para K-Means
silueta_km = silhouette_score(x_scaled, clusters_km.labels_)
davis_bouldin_km = davies_bouldin_score(x_scaled, clusters_km.labels_)


1.5691728523526178

In [8]:
silueta_km

0.2505734481247145

In [9]:
davis_bouldin_km

1.5691728523526178